# Install pipelines SDK

#### Please wait till this cell completes and then run next cells. This just need to be run once per active kernel.

In [1]:
%env RELEASE_VERSION=1.0.0
!pip install https://storage.googleapis.com/ml-pipeline/release/${RELEASE_VERSION}/kfp.tar.gz --upgrade

env: RELEASE_VERSION=1.0.0
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 122 kB 4.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.5 MB 14.1 MB/s eta 0:00:01
  Created wheel for kfp: filename=kfp-1.0.0-py3-none-any.whl size=168415 sha256=e052704fc98177443811a8f885a074a27c630e254ff809025dfea4d6698bdff7
  Stored in directory: /tmp/pip-ephem-wheel-cache-ul3ksxv6/wheels/4b/7c/e8/0e2908409b03b0bc529b8f927b57f44f01630c4b508d321d6b
Successfully built kfp
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


# Import kfp pkgs

In [2]:
import os
import kfp
import kfp.dsl as dsl
import kfp.compiler as compiler
from kubernetes import client as k8s_client

# List existing pipeline experiments

In [3]:
existing_token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
client = kfp.Client(existing_token=existing_token)

In [4]:
client.list_experiments()

{'experiments': [{'created_at': datetime.datetime(2021, 9, 5, 13, 34, 40, tzinfo=tzlocal()),
                  'description': None,
                  'id': '609fee28-4a55-414e-90a4-456591d1de63',
                  'name': 'Default',
                  'resource_references': [{'key': {'id': 'ocdkube',
                                                   'type': 'NAMESPACE'},
                                           'name': None,
                                           'relationship': 'OWNER'}],
                  'storage_state': 'STORAGESTATE_AVAILABLE'}],
 'next_page_token': None,
 'total_size': 1}

# Create a Dkube Storage experiment

In [5]:
dkube_store_experiment = client.create_experiment(name='Dkube - Storage pl')

# Define e2e Storage Pipeline with Dkube components

In [11]:
import kfp.dsl as dsl
from kfp import components
from kubernetes import client as k8s_client

import os
import json
from random import randint

components_url = "/mnt/dkube/pipeline/components/"
dkube_storage_op            = components.load_component_from_file(components_url + "/storage/component.yaml")

@dsl.pipeline(
    name='storage',
    description='sample pipeline with dkube storage component'
)
def d3pipeline(
    #Dkube authentication token
    auth_token = os.getenv("DKUBE_USER_ACCESS_TOKEN"),
    #Namespace to export the datasets
    namespace = os.getenv("USERNAME"),
    command = 'export',
    #List of artifacts to export, format : <claimname>@<class>://<owner>:<datumname>/<version>
    #Class must be one of: [dataset, model, program]
    #To get the latest version use "latest" in place of <version>, otherwise use v1, v2, etc. (For program, use full commitid)
    #(Default version is "latest")
    input_volumes = json.dumps(["mn-proj-3@program://mnist"]),
    output_volumes = json.dumps(["mn-data@dataset://mnist/v1"]),
    intermediate_volume = 'temp-1'
    ):

    
    store       = dkube_storage_op(command, auth_token, namespace, input_volumes=input_volumes,
                                   output_volumes=output_volumes, intermediate_volume=intermediate_volume)
#                                  )                                   

# Compile and generate tar ball

In [12]:
compiler.Compiler().compile(d3pipeline, 'dkube_storage_pl.tar.gz')
# Upload this generated tarball into the Pipelines UI

# Create and Run Pipeline

#### Click the dkube-training stage to see the enhanced Dkube Datascience dashboard, metrics and graphs. Click the dkube-viewer stage for the simple UI to test the model predecitions

#### <font color=red> Fill the training_program, training_dataset and training_output_model with appropriate values in the next cell </font>

In [13]:
run = client.run_pipeline(dkube_store_experiment.id, 'dkube_storage_pipeline_1', 'dkube_storage_pl.tar.gz', params={})

# Define Reclaim Pipeline with Dkube components

In [14]:
@dsl.pipeline(
    name='reclaim',
    description='sample pipeline with dkube storage - reclaim component'
)
def d3_reclaim_pipeline(
    command = 'reclaim',
    #Dkube authentication token
    auth_token = os.getenv("DKUBE_USER_ACCESS_TOKEN"),
    #Namespace of volumes to reclaim
    namespace = os.getenv("USERNAME"),
    # UID relating all volumes
    uid = '5c8a65dc-0268-4caf-833d-f123c9fe13d9'
    ):

    store       = dkube_storage_op(command, auth_token, namespace, uid=uid)

In [15]:
compiler.Compiler().compile(d3_reclaim_pipeline, 'dkube_storage_reclaim_pl.tar.gz')
# Upload this generated tarball into the Pipelines UI

In [16]:
run = client.run_pipeline(dkube_store_experiment.id, 'dkube_storage_reclaim_pipeline_1', 'dkube_storage_reclaim_pl.tar.gz', params={})